## Classifying ECG Images using CNN

# 1. Import Necessary Libraries

In [15]:
import numpy as np                                                   
import tensorflow                                                    
from tensorflow.keras.models import Sequential                        
from tensorflow.keras import layers                  
from tensorflow.keras.layers import Dense,Flatten
from tensorflow.keras.layers import Conv2D,MaxPooling2D                 
from keras.preprocessing.image import ImageDataGenerator                                                       

# 2. Image Data Augementation

In [16]:
train_datagen=ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)  #training

In [17]:
test_datagen=ImageDataGenerator(rescale=1./255) #testing

# 3. Loading our data and performing data agumentation

In [18]:
x_train=train_datagen.flow_from_directory(r'C:\Users\Kkira\OneDrive\Desktop\mini project\ECG Arrhythmia\train'
                                          ,target_size=(64,64),batch_size=32,class_mode='categorical')

x_test=test_datagen.flow_from_directory(r'C:\Users\Kkira\OneDrive\Desktop\mini project\ECG Arrhythmia\test'
                                        ,target_size=(64,64),batch_size=32,class_mode='categorical')

Found 15341 images belonging to 6 classes.
Found 6825 images belonging to 6 classes.


In [19]:
print(x_train.class_indices) 

{'Left Bundle Branch Block': 0, 'Normal': 1, 'Premature Atrial Contraction': 2, 'Premature Ventricular Contractions': 3, 'Right Bundle Branch Block': 4, 'Ventricular Fibrillation': 5}


In [20]:
from collections import Counter as c
c(x_train.labels)

Counter({0: 504, 1: 7346, 2: 2054, 3: 2759, 4: 2239, 5: 439})

# 4. Creating the model

In [21]:
# create model
model=Sequential()
# adding model layer
model.add(Conv2D(32,(3,3),input_shape=(64,64,3),activation='relu'))#convolutional layer
model.add(MaxPooling2D(pool_size=(2,2))) #MaxPooling2D-for downsampling the input

model.add(Conv2D(32,(3,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())#flatten the dimension of the image
model.add(Dense(32))#deeply connected neural network layers.
model.add(Dense(6,activation='softmax'))#output layer with 6 neurons

In [22]:
model.summary()#summary of our model

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 32)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_3 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 6272)              0         
                                                                 
 dense_2 (Dense)             (None, 32)               

# 5. Compiling the model

In [23]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

# 6. Fitting the model

In [24]:
# Fit the model
model.fit_generator(generator=x_train,steps_per_epoch = len(x_train),
                    epochs=10, validation_data=x_test,validation_steps = len(x_test))

C:\Users\Kkira\AppData\Local\Temp\ipykernel_592\1433457599.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(generator=x_train,steps_per_epoch = len(x_train),


Epoch 1/10
480/480 [==============================] - 401s 831ms/step - loss: 0.9343 - accuracy: 0.6769 - val_loss: 0.5611 - val_accuracy: 0.8188
Epoch 2/10
480/480 [==============================] - 267s 555ms/step - loss: 0.3126 - accuracy: 0.9063 - val_loss: 0.4778 - val_accuracy: 0.8418
Epoch 3/10
480/480 [==============================] - 243s 506ms/step - loss: 0.2637 - accuracy: 0.9227 - val_loss: 0.4564 - val_accuracy: 0.8577
Epoch 4/10
480/480 [==============================] - 238s 495ms/step - loss: 0.2264 - accuracy: 0.9359 - val_loss: 0.3337 - val_accuracy: 0.8866
Epoch 5/10
480/480 [==============================] - 241s 502ms/step - loss: 0.1901 - accuracy: 0.9443 - val_loss: 0.3072 - val_accuracy: 0.9020
Epoch 6/10
480/480 [==============================] - 242s 504ms/step - loss: 0.1771 - accuracy: 0.9472 - val_loss: 0.3214 - val_accuracy: 0.8905
Epoch 7/10
480/480 [==============================] - 240s 501ms/step - loss: 0.1632 - accuracy: 0.9510 - val_loss: 0.3096 -

# 7. Saving our Model

In [25]:
model.save('ECG.h5')

# 8. Result Prediction

In [26]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("ECG.h5") 

In [27]:
img = tensorflow.keras.utils.load_img(r"C:\Users\Kkira\OneDrive\Desktop\mini project\ECG Arrhythmia\ECG evaluation\lbbb.png",target_size= (64,64))#loading of the image
x = tensorflow.keras.utils.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict(x)
classes_x=np.argmax(pred,axis=1)#predicting the classes
classes_x

1/1 [==============================] - 0s 312ms/step


array([0], dtype=int64)

In [28]:
index=['Left Bundle Branch Block','Normal','Premature Atrial Contraction',
       'Premature Ventricular Contractions', 'Right Bundle Branch Block','Ventricular Fibrillation']
result=str(index[classes_x[0]])
result

'Left Bundle Branch Block'

In [29]:
img = tensorflow.keras.utils.load_img(r"C:\Users\Kkira\OneDrive\Desktop\mini project\ECG Arrhythmia\ECG evaluation\normal.png",target_size= (64,64))#loading of the image
x = tensorflow.keras.utils.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict(x)
classes_x=np.argmax(pred,axis=1)#predicting the classes
classes_x

1/1 [==============================] - 0s 47ms/step


array([1], dtype=int64)

In [30]:
index=['Left Bundle Branch Block','Normal','Premature Atrial Contraction',
       'Premature Ventricular Contractions', 'Right Bundle Branch Block','Ventricular Fibrillation']
result=str(index[classes_x[0]])
result

'Normal'

In [31]:
img = tensorflow.keras.utils.load_img(r"C:\Users\Kkira\OneDrive\Desktop\mini project\ECG Arrhythmia\ECG evaluation\pac.png",target_size= (64,64))#loading of the image
x = tensorflow.keras.utils.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict(x)
classes_x=np.argmax(pred,axis=1)#predicting the classes
classes_x

1/1 [==============================] - 0s 47ms/step


array([2], dtype=int64)

In [32]:
img = tensorflow.keras.utils.load_img(r"C:\Users\Kkira\OneDrive\Desktop\mini project\ECG Arrhythmia\ECG evaluation\pvc.png",target_size= (64,64))#loading of the image
x = tensorflow.keras.utils.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict(x)
classes_x=np.argmax(pred,axis=1)#predicting the classes
classes_x

1/1 [==============================] - 0s 47ms/step


array([3], dtype=int64)

In [33]:
index=['Left Bundle Branch Block','Normal','Premature Atrial Contraction',
       'Premature Ventricular Contractions', 'Right Bundle Branch Block','Ventricular Fibrillation']
result=str(index[classes_x[0]])
result

'Premature Ventricular Contractions'

In [34]:
img = tensorflow.keras.utils.load_img(r"C:\Users\Kkira\OneDrive\Desktop\mini project\ECG Arrhythmia\ECG evaluation\rbbb.png",target_size= (64,64))#loading of the image
x = tensorflow.keras.utils.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict(x)
classes_x=np.argmax(pred,axis=1)#predicting the classes
classes_x

1/1 [==============================] - 0s 47ms/step


array([4], dtype=int64)

In [35]:
index=['Left Bundle Branch Block','Normal','Premature Atrial Contraction',
       'Premature Ventricular Contractions', 'Right Bundle Branch Block','Ventricular Fibrillation']
result=str(index[classes_x[0]])
result

'Right Bundle Branch Block'

In [36]:
img = tensorflow.keras.utils.load_img(r"C:\Users\Kkira\OneDrive\Desktop\mini project\ECG Arrhythmia\ECG evaluation\vf.png",target_size= (64,64))#loading of the image
x = tensorflow.keras.utils.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict(x)
classes_x=np.argmax(pred,axis=1)#predicting the classes
classes_x

1/1 [==============================] - 0s 62ms/step


array([5], dtype=int64)

In [37]:
index=['Left Bundle Branch Block','Normal','Premature Atrial Contraction',
       'Premature Ventricular Contractions', 'Right Bundle Branch Block','Ventricular Fibrillation']
result=str(index[classes_x[0]])
result

'Ventricular Fibrillation'